# Classification Task Results

In [2]:
from sklearn.metrics import precision_recall_fscore_support
from difflib import SequenceMatcher
from scripts.config import DefaultArgsNamespace
import pandas as pd
import numpy as np
import os
import seaborn as sns
from sklearn.metrics import average_precision_score, precision_recall_curve

from metrics.metrics import *

In [3]:
args = DefaultArgsNamespace()

results_csv = './results/model_id_65531725_on_20241031-124604/preds.csv'
# path to results csv
results_dir = results_csv.rsplit('/', 1)[0]


# Preprocessing

In [ ]:
# get keysteps dict
keysteps_dict = args.dataloader_params['keysteps']
# Inverting the dictionary to map keystep_id to natural language description
keystep_id_to_desc = {i: v for i, v in enumerate(keysteps_dict.values())}

keysteps_dict = keystep_id_to_desc


In [ ]:
df = pd.read_csv(results_csv)

keysteps_dict = args.dataloader_params['keysteps']
print(keysteps_dict)

# Inverting the dictionary to map keystep_id to natural language description
keystep_id_to_desc = {i: v for i, v in enumerate(keysteps_dict.values())}

# Adding a new column to the dataframe that maps keystep_id to the corresponding natural language description
df['keystep_description'] = df['keystep_id'].map(keystep_id_to_desc)
df['pred_keystep_description'] = df['pred_keystep_id'].map(keystep_id_to_desc)

# Display the updated dataframe
df.head()

# Generate metrics

In [ ]:
# Grouping by subject_id and trial_id
results = []

# Assuming 'results_csv' is defined earlier and points to the path of the results CSV
results_dir = results_csv.rsplit('/', 1)[0]


all_accuracy = []
all_edit_distances = []
all_edit_scores = []
all_precisions = []
all_recalls = []
all_f1_k = []

all_ground_truth = []
all_predicted = []

for (subject, trial), group in df.groupby(['subject_id', 'trial_id']):
    print("*" * 50)
    print(f"Subject: {subject}, Trial: {trial}")
    
    # Extract ground truth and predicted keysteps
    ground_truth = group['keystep_id'].tolist()  # Ground truth per frame
    predicted = group['pred_keystep_id'].tolist()  # Predicted per frame
    

    # Calculate edit distance
    edit_distance = calculate_edit_distance(ground_truth, predicted)
    # Calculate edit score
    edit_score = calculate_edit_score(ground_truth, predicted, edit_distance)


    all_edit_distances.append(edit_distance)
    all_edit_scores.append(edit_score)

    print("edit_distance: ", edit_distance)
    print("edit_score: ", edit_score)

    all_ground_truth.append(ground_truth)
    all_predicted.append(predicted)

    f1_aggregated_metrics = {}

    f1_score, precision, recall = calculate_f1_classification(ground_truth, predicted)
    # Store precision, recall, f1 metrics for each threshold
    metrics = {
        f'precision': precision,
        f'recall': recall,
        f'f1': f1_score,
    }
    # Append precision, recall, and f1 to the respective lists
    all_precisions.append(precision)
    all_recalls.append(recall)
    all_f1_k.append(f1_score)


    ## Calculate accuracy
    accuracy = np.mean(np.array(ground_truth) == np.array(predicted))
    print("accuracy: ", accuracy)
    
    all_accuracy.append(accuracy)

    # Append ground truth and predicted to global lists
    all_ground_truth.append(ground_truth)
    all_predicted.append(predicted)

    # Append result for this subject and trial
    results.append({
        'subject_id': subject,
        'trial_id': trial,
        'accuracy': accuracy,
        'edit_distance': edit_distance,
        'edit_score': edit_score,
        'f1': f1_score,
        'precision': precision,
        'recall': recall,
        'ground_truth': ground_truth,
        'predicted': predicted,
    })



### Save results to csvs

In [ ]:
# Save the results to CSV
results_df = pd.DataFrame(results)
results_df.to_csv(f'{results_dir}/subject_wise_results.csv', index=False)

# Calculate average metrics
average_accuracy = np.mean(all_accuracy)
average_edit_distance = int(np.mean(all_edit_distances))
average_edit_score = np.mean(all_edit_scores)

# Calculate the average precision, recall, and f1 for each IoU threshold
average_precisions = {f'average_precision': np.mean(all_precisions)}
average_recalls = {f'average_recall': np.mean(all_recalls)}
average_f1_k = {f'average_f1': np.mean(all_f1_k)}

# Prepare the dictionary for average metrics
average_metrics = {
    'accuracy': average_accuracy,
    'edit_distance': average_edit_distance,
    'edit_score': average_edit_score,
}
average_metrics.update(average_precisions)
average_metrics.update(average_recalls)
average_metrics.update(average_f1_k)

# Convert the average metrics dictionary to a DataFrame and save it as CSV
# Define the desired order of columns
column_order = ['average_precision', 'average_recall', 'average_f1', 'accuracy', 'edit_score', 'edit_distance']
# Convert the average metrics dictionary to a DataFrame and reorder columns
average_metrics_df = pd.DataFrame(average_metrics, index=[0])[column_order]

average_metrics_df.to_csv(f'{results_dir}/average_metrics.csv', index=False)

all_ground_truth = np.concatenate(all_ground_truth)
all_predicted = np.concatenate(all_predicted)
unique_actions = np.unique(np.union1d(all_predicted, all_ground_truth))
print("Unique actions: ", unique_actions)

# Calculate final class-wise accuracy across all trials
final_classwise_accuracy = calculate_final_classwise_accuracy(all_ground_truth, all_predicted, unique_actions)
print("Final Class-wise Accuracy:")
for action, accuracy in final_classwise_accuracy.items():
    print(f"{action}: {accuracy}")

# Save the final class-wise accuracy to CSV
classwise_accuracy_df = pd.DataFrame.from_dict(final_classwise_accuracy, orient='index', columns=['accuracy'])
classwise_accuracy_csv_path = f'{results_dir}/final_classwise_accuracy.csv'
classwise_accuracy_df.to_csv(classwise_accuracy_csv_path, index_label='action')


# Plotting

### Helper functions

In [ ]:
import matplotlib.cm as cm


# Function to generate a color map for keystep IDs with distinct colors, excluding red shades
def get_unique_colors(num_colors):
    # Use a base colormap (e.g., hsv)
    cmap = cm.get_cmap('tab20')  # Get 'num_colors' evenly spaced colors from the colormap
    
    # Generate the list of colors by sampling the colormap
    colors = [cmap(i / num_colors) for i in range(num_colors)]
    return colors

# get keysteps dict
keysteps_dict = args.dataloader_params['keysteps']
# Inverting the dictionary to map keystep_id to natural language description
keystep_id_to_desc = {i: v for i, v in enumerate(keysteps_dict.values())}

keysteps_dict = keystep_id_to_desc

# Get unique colors for the number of keysteps
num_keysteps = len(keysteps_dict)
colors = get_unique_colors(num_keysteps)

# Create a color dictionary mapping keystep IDs to colors
keystep_color_dict = {keystep_id: colors[i] for i, keystep_id in enumerate(keysteps_dict.keys())}

# Output the dictionary for reference
keystep_color_dict


# Plot Keysteps

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import matplotlib.cm as cm

# Function to generate a color map for keystep IDs with distinct colors
def get_unique_colors(num_colors):
    return cm.get_cmap('tab20', num_colors)  # Using 'hsv' for distinct colors

# Function to plot keystep sequences with unique colors and a legend at the bottom
def plot_keystep_sequences(ground_truth, predicted, subject, trial, keystep_dict, plots_dir):
    print("Plotting keystep sequences")
    print(ground_truth)
    print(predicted)
    # Get unique keystep IDs for ground truth and predicted keysteps
    all_keysteps = list(set(ground_truth['keystep_id'].unique()).union(set(predicted['pred_keystep_id'].unique())))
    print((all_keysteps))
    # Generate a color map with as many unique colors as there are keysteps
    color_map = get_unique_colors(len(all_keysteps))
    color_dict = {keystep_id: color_map(i) for i, keystep_id in enumerate(all_keysteps)}
    print("color duict", color_dict)

    # Create figure and axis
    fig, ax = plt.subplots(figsize=(14, 5))  # Increased height and width for better spacing

    # Plot ground truth keysteps
    y_pos_gt = 0  # Position for ground truth timeline
    for i, (start, end, keystep_id) in enumerate(zip(ground_truth['start_frame'], ground_truth['end_frame'], ground_truth['keystep_id'])):
        width = max(end - start, 5)  # Ensure a minimum width for short keysteps
        rect = patches.Rectangle((start, y_pos_gt), width, 1, edgecolor=color_dict[keystep_id], facecolor=color_dict[keystep_id], linewidth=2, alpha=0.85)
        ax.add_patch(rect)

    # Plot predicted keysteps with mismatch highlighting
    y_pos_pred = 2  # Increased vertical space between ground truth and prediction
    for i, (start, end, pred_keystep_id) in enumerate(zip(predicted['start_frame'], predicted['end_frame'], predicted['pred_keystep_id'])):
        width = max(end - start, 5)  # Ensure a minimum width for short keysteps
        gt_keystep_id = ground_truth['keystep_id'].iloc[i] if i < len(ground_truth) else None
        if pred_keystep_id != gt_keystep_id:
            # Mismatch: use thicker red boundary
            rect = patches.Rectangle((start, y_pos_pred), width, 1, edgecolor='black', facecolor=color_dict[pred_keystep_id], linewidth=3, alpha=0.85)
            # print("Mismatch")
            # print(f"Predicted: {pred_keystep_id}, Ground Truth: {gt_keystep_id}, Start: {start}, End: {end}")
        else:
            # Match: use normal boundary color
            rect = patches.Rectangle((start, y_pos_pred), width, 1, edgecolor=color_dict[pred_keystep_id], facecolor=color_dict[pred_keystep_id], linewidth=2, alpha=0.85)
        ax.add_patch(rect)

    # Add labels for the timelines
    ax.text(-300, y_pos_gt + 0.5, 'Ground Truth', va='center', fontsize=10, color='blue')
    ax.text(-300, y_pos_pred + 0.5, 'Prediction', va='center', fontsize=10, color='red')

    # Create a horizontal legend at the bottom
    legend_elements = [patches.Patch(facecolor=color_dict[k], edgecolor=color_dict[k], label=f"{k}: {keystep_dict[k]}") for k in all_keysteps]
    ax.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, -0.15), fancybox=True, shadow=True, ncol=4, title="Keysteps")

    # Set axis limits and labels
    ax.set_xlim([0, max(ground_truth['end_frame'].max(), predicted['end_frame'].max())])
    ax.set_ylim([-1, 4])
    ax.set_xlabel('Frame Number')
    ax.set_title(f'Keystep Sequences for Subject {subject} Trial {trial}')

    # Hide y-axis
    ax.get_yaxis().set_visible(False)

    # hide top and right spines
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)

    # save figure
    plt.savefig(f'{plots_dir}/subject_{subject}_trial_{trial}_keystep_sequence.png', bbox_inches='tight')
    # # Show plot
    # plt.tight_layout()
    # plt.show()




### Bar plot Horizontal

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# Function to generate a color map for keystep IDs with distinct colors
def get_unique_colors(num_colors):
    return cm.get_cmap('tab20', num_colors)  # Using 'hsv' for distinct colors

# Function to plot keystep sequences using horizontal bar plot
def plot_keystep_bars(ground_truth, predicted, subject, trial, keystep_dict, plots_dir):
    # Get unique keystep IDs for ground truth and predicted keysteps
    all_keysteps = list(set(ground_truth['keystep_id'].unique()).union(set(predicted['pred_keystep_id'].unique())))

    # Generate a color map with as many unique colors as there are keysteps
    color_map = get_unique_colors(len(all_keysteps))
    color_dict = {keystep_id: color_map(i) for i, keystep_id in enumerate(all_keysteps)}

    # Create figure and axis
    fig, ax = plt.subplots(figsize=(10, 6))  # Increased height for better spacing

    # Plot horizontal bars for ground truth keysteps
    y_pos_gt = 0  # Position for ground truth timeline
    for i, (start, end, keystep_id) in enumerate(zip(ground_truth['start_frame'], ground_truth['end_frame'], ground_truth['keystep_id'])):
        width = max(end - start, 5)  # Ensure a minimum width of 5 for visibility
        ax.barh(y_pos_gt, width, left=start, height=0.9, color=color_dict[keystep_id], edgecolor='black',  linewidth=0.25)
        y_pos_gt += 1  # Move to next row for ground truth

    # Plot horizontal bars for predicted keysteps
    y_pos_pred = y_pos_gt + 1  # Leave a gap between ground truth and predicted
    for i, (start, end, pred_keystep_id) in enumerate(zip(predicted['start_frame'], predicted['end_frame'], predicted['pred_keystep_id'])):
        width = max(end - start, 5)  # Ensure a minimum width of 5 for visibility
        if pred_keystep_id != ground_truth['keystep_id'].iloc[i]:
            ax.barh(y_pos_pred, width, left=start, height=0.9, color=color_dict[pred_keystep_id], edgecolor='black', linewidth=2, hatch='//')  # Highlight mismatches
            # ax.barh(y_pos_pred, width, left=start, height=0.6, color=color_dict[pred_keystep_id], edgecolor='black', linewidth=2)  # Highlight mismatches

        else:
            ax.barh(y_pos_pred, width, left=start, height=0.9, color=color_dict[pred_keystep_id], edgecolor='black', linewidth=0.25)
        y_pos_pred += 1  # Move to next row for predicted

    # Create a horizontal legend at the bottom
    legend_elements = [plt.Line2D([0], [0], color=color_dict[k], lw=4, label=f"{k}: {keystep_dict[k]}") for k in all_keysteps]
    ax.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, -0.15), fancybox=True, shadow=True, ncol=4, title="Keysteps", fontsize=18,  prop={'weight': 'bold'})

    # Add labels for the timelines
    ax.text(-300, y_pos_gt - 7, 'Ground Truth', va='center', fontsize=14, color='blue')
    ax.text(-300, y_pos_pred - 6, 'Prediction', va='center', fontsize=14, color='red')

    # hide top and right spines
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)


    # Set axis limits and labels
    ax.set_xlim([0, max(ground_truth['end_frame'].max(), predicted['end_frame'].max())])
    ax.set_yticks([])  # Hide y-axis labels
    ax.set_xlabel('Frame Number')
    ax.set_title(f'Keystep Sequences for Subject {subject} Trial {trial}')

    # set font size for x-axis
    plt.xticks(fontsize=14)

    

    # save figure
    plt.savefig(f'{plots_dir}/subject_{subject}_trial_{trial}_keystep_bars.png', bbox_inches='tight', dpi=600)
    plt.savefig(f'{plots_dir}/subject_{subject}_trial_{trial}_keystep_bars.pdf', bbox_inches='tight', dpi=600)
    # Show plot
    plt.tight_layout()
    plt.show()


### Plot keysteps sequence in a single timeline

In [ ]:

# plots_dir = f'{results_dir}/plots'
# os.makedirs(plots_dir, exist_ok=True)

# # Example usage with one subject's trial data
# for (subject, trial), group in df.groupby(['subject_id', 'trial_id']):
#     ground_truth_data = group[['keystep_id', 'start_frame', 'end_frame']]
#     predicted_data = group[['pred_keystep_id', 'start_frame', 'end_frame']]

#     plots_dir = f'{results_dir}/plots/sequence_plots'
#     os.makedirs(plots_dir, exist_ok=True)
#     # save figure
#     # Plot using the keystep IDs with unique colors, thicker boundaries, and a horizontal legend
#     plot_keystep_sequences(ground_truth_data, predicted_data, subject, trial, keysteps_dict, plots_dir)
#     break  # Only plot the first subject's trial for now

### Plot bar stacked timeline

In [ ]:


df = pd.read_csv(results_csv)

keysteps_dict = args.dataloader_params['keysteps']
print(keysteps_dict)

# Inverting the dictionary to map keystep_id to natural language description
keystep_id_to_desc = {i: v for i, v in enumerate(keysteps_dict.values())}

# Adding a new column to the dataframe that maps keystep_id to the corresponding natural language description
df['keystep_description'] = df['keystep_id'].map(keystep_id_to_desc)
df['pred_keystep_description'] = df['pred_keystep_id'].map(keystep_id_to_desc)

# Display the updated dataframe
df.head()

# get keysteps dict
keysteps_dict = args.dataloader_params['keysteps']
# Inverting the dictionary to map keystep_id to natural language description
keystep_id_to_desc = {i: v for i, v in enumerate(keysteps_dict.values())}

keysteps_dict = keystep_id_to_desc


i = 0
# Example usage with one subject's trial data
for (subject, trial), group in df.groupby(['subject_id', 'trial_id']):
    ground_truth_data = group[['keystep_id', 'start_frame', 'end_frame']]
    predicted_data = group[['pred_keystep_id', 'start_frame', 'end_frame']]

    plots_dir = f'{results_dir}/plots/bar_plots'
    os.makedirs(plots_dir, exist_ok=True)
    # Plot using the keystep IDs with horizontal bar plot and color legend
    plot_keystep_bars(ground_truth_data, predicted_data, subject, trial, keysteps_dict, plots_dir)
    # break  # Only plot the first subject's trial for now

    i += 1

    if (i == 3):
        break
    # break


## Detail result plot


In [ ]:
# plot average metrics (f1, precision, recall)
average_metrics_df = pd.read_csv(f'{results_dir}/average_metrics.csv')

# Plot the average metrics
fig, ax = plt.subplots(figsize=(8, 5))

# Plot the average precision, recall, and f1 scores
average_metrics_df[['average_precision', 'average_recall', 'average_f1']].plot(kind='bar', ax=ax, color=['blue', 'green', 'orange'])


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
# Sample data from the table
data = {
    'Model': ['ICRA Model']*18,
    'Modality': ['Video (Resnet50)']*6 + ['I3D (RGB,Flow)']*6 + ['IMU (Smartwatch)']*6,
    'Window': ['4s']*3 + ['Full']*3 + ['4s']*3 + ['Full']*3 + ['4s']*3 + ['Full']*3,
    'IOU': ['0', '0.25', '0.5']*6,
    'Precision': [0.658, 0.658, 0.567, 0.644, 0.640, 0.625, 
                  0.663, 0.663, 0.636, 0.661, 0.661, 0.651, 
                  0.556, 0.556, 0.528, 0.490, 0.490, 0.457],
    'Recall': [0.701, 0.701, 0.594, 0.683, 0.670, 0.651, 
               0.715, 0.715, 0.674, 0.703, 0.703, 0.687, 
               0.614, 0.614, 0.584, 0.551, 0.551, 0.507],
    'F1-score': [0.659, 0.659, 0.565, 0.641, 0.634, 0.619, 
                 0.672, 0.672, 0.640, 0.662, 0.662, 0.651, 
                 0.571, 0.571, 0.542, 0.500, 0.500, 0.464],
    'Accuracy': [0.700, 0.700, 0.700, 0.680, 0.680, 0.680,
                 0.711, 0.711, 0.711, 0.701, 0.701, 0.701,
                 0.603, 0.603, 0.603, 0.562, 0.562, 0.562],
    'Edit Score': [0.862, 0.862, 0.862, 0.798, 0.798, 0.798,
                   0.865, 0.865, 0.865, 0.810, 0.810, 0.810,
                   0.839, 0.839, 0.839, 0.753, 0.753, 0.753],
    'Edit Distance': [4.909, 4.909, 4.909, 3.955, 3.955, 3.955,
                      4.909, 4.909, 4.909, 3.682, 3.682, 3.682,
                      5.591, 5.591, 5.591, 4.591, 4.591, 4.591]
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Set plot style
sns.set_theme(style="whitegrid", palette=sns.color_palette("hls", 3))


# Plot 1: Precision for each modality and window size
plt.figure(figsize=(10, 6))
sns.lineplot(data=df, x='IOU', y='Precision', hue='Modality', style='Window', markers=True, linewidth=1.5, markersize=10)
plt.title('Precision vs IOU')
plt.xlabel('IOU')
plt.ylabel('Precision')
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=7)
plt.savefig(f'{results_dir}/plots/precision_vs_iou.png', bbox_inches='tight')
plt.tight_layout()
plt.show()

# Plot 2: Recall for each modality and window size
plt.figure(figsize=(10, 6))
sns.lineplot(data=df, x='IOU', y='Recall', hue='Modality', style='Window', markers=True)
plt.title('Recall vs IOU')
plt.xlabel('IOU')
plt.ylabel('Recall')
plt.legend( loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=7)
plt.savefig(f'{results_dir}/plots/recall_vs_iou.png', bbox_inches='tight')
plt.tight_layout()
plt.show()

# Plot 3: F1-score for each modality and window size
plt.figure(figsize=(10, 6))
sns.lineplot(data=df, x='IOU', y='F1-score', hue='Modality', style='Window', markers=True)
plt.title('F1-score vs IOU')
plt.xlabel('IOU')
plt.ylabel('F1-score')
plt.legend( loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=7)
plt.savefig(f'{results_dir}/plots/f1_score_vs_iou.png', bbox_inches='tight')
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Sample data from the table
data = {
    'Model': ['ICRA Model']*18,
    'Modality': ['Video (Resnet50)']*6 + ['I3D (RGB,Flow)']*6 + ['IMU (Smartwatch)']*6,
    'Window': ['4s']*3 + ['Full']*3 + ['4s']*3 + ['Full']*3 + ['4s']*3 + ['Full']*3,
    'IOU': ['0', '0.25', '0.5']*6,
    'Precision': [0.658, 0.658, 0.567, 0.644, 0.640, 0.625, 
                  0.663, 0.663, 0.636, 0.661, 0.661, 0.651, 
                  0.556, 0.556, 0.528, 0.490, 0.490, 0.457],
    'Recall': [0.701, 0.701, 0.594, 0.683, 0.670, 0.651, 
               0.715, 0.715, 0.674, 0.703, 0.703, 0.687, 
               0.614, 0.614, 0.584, 0.551, 0.551, 0.507],
    'F1-score': [0.659, 0.659, 0.565, 0.641, 0.634, 0.619, 
                 0.672, 0.672, 0.640, 0.662, 0.662, 0.651, 
                 0.571, 0.571, 0.542, 0.500, 0.500, 0.464],
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Set plot style
sns.set_theme(style="whitegrid", palette=sns.color_palette("hls", 3))

# Create a figure with three subplots horizontally
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# Plot 1: Precision vs IOU
sns.lineplot(data=df, x='IOU', y='Precision', hue='Modality', style='Window', markers=True, linewidth=1.5, markersize=10, ax=axes[0])
axes[0].set_title('Precision vs IOU')
axes[0].set_xlabel('IOU')
axes[0].set_ylabel('Precision')

# Plot 2: Recall vs IOU
sns.lineplot(data=df, x='IOU', y='Recall', hue='Modality', style='Window', markers=True, linewidth=1.5, markersize=10, ax=axes[1])
axes[1].set_title('Recall vs IOU')
axes[1].set_xlabel('IOU')
axes[1].set_ylabel('Recall')

# Plot 3: F1-score vs IOU
sns.lineplot(data=df, x='IOU', y='F1-score', hue='Modality', style='Window', markers=True, linewidth=1.5, markersize=10, ax=axes[2])
axes[2].set_title('F1-score vs IOU')
axes[2].set_xlabel('IOU')
axes[2].set_ylabel('F1-score')

# Remove legends from individual plots
axes[0].legend_.remove()
axes[1].legend_.remove()
axes[2].legend_.remove()

# Create a single legend for the entire figure
handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=8)

plt.savefig(f'{results_dir}/plots/metrics_vs_iou.png', bbox_inches='tight')

# Adjust layout
plt.tight_layout(rect=[0, 0, 1, 0.95])  # Leave space for the legend
plt.show()



# Subject - Trial Keystep Classification Visualization

In [ ]:
import os
import pandas as pd

# Assuming results_csv and results_dir are defined
df = pd.read_csv(results_csv)

# Load keysteps_dict from args
keysteps_dict = args.dataloader_params['keysteps']

# Invert dictionary for keystep descriptions
keystep_id_to_desc = {i: v for i, v in enumerate(keysteps_dict.values())}

# Map descriptions to DataFrame
df['keystep_description'] = df['keystep_id'].map(keystep_id_to_desc)
df['pred_keystep_description'] = df['pred_keystep_id'].map(keystep_id_to_desc)

# Iterate through groups and create CSV for each subject and trial
for (subject, trial), group in df.groupby(['subject_id', 'trial_id']):
    # Combine ground truth and predicted data
    result_data = pd.DataFrame({
        'groundtruth_keystep_id': group['keystep_id'],
        'groundtruth_start_frame': group['start_frame'],
        'groundtruth_end_frame': group['end_frame'],
        'predicted_keystep_id': group['pred_keystep_id'],
        'predicted_start_frame': group['start_frame'],
        'predicted_end_frame': group['end_frame']
    })

    # Define the output directory and filename
    subject_results_dir = os.path.join(results_dir, 'subject_specific_results')
    os.makedirs(subject_results_dir, exist_ok=True)
    
    csv_filename = f'{subject}_{trial}.csv'
    csv_path = os.path.join(subject_results_dir, csv_filename)

    # Save the data to CSV
    result_data.to_csv(csv_path, index=False)

    print(f"Saved {csv_path}")


In [10]:
import cv2
import pandas as pd
import os

def visualize_keysteps_from_csv(video_path, csv_path, output_path, keysteps_dict):
    # Load the CSV file
    df = pd.read_csv(csv_path)
    
    if df.empty:
        print(f"No data found in {csv_path}.")
        return
    
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Unable to open video {video_path}")
        return
    
    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Define the video writer for saving the output
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    # Frame index
    frame_idx = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  # End of video
        
        # Collect all ground truth and predicted keysteps for the current frame
        groundtruth_keysteps = []
        predicted_keysteps = []
        
        for _, row in df.iterrows():
            # Ground truth keystep
            if row['groundtruth_start_frame'] <= frame_idx <= row['groundtruth_end_frame']:
                gt_text = keysteps_dict.get(row['groundtruth_keystep_id'], f"Unknown ({row['groundtruth_keystep_id']})")
                groundtruth_keysteps.append(gt_text)
            
            # Predicted keystep
            if row['predicted_start_frame'] <= frame_idx <= row['predicted_end_frame']:
                pred_text = keysteps_dict.get(row['predicted_keystep_id'], f"Unknown ({row['predicted_keystep_id']})")
                predicted_keysteps.append(pred_text)
        
        # Determine text and colors
        gt_text = f"GT: {', '.join(groundtruth_keysteps)}" if groundtruth_keysteps else None
        pred_text = f"Pred: {', '.join(predicted_keysteps)}" if predicted_keysteps else None
        color = (0, 255, 0) if set(groundtruth_keysteps) == set(predicted_keysteps) else (0, 0, 255)
        
        # Draw text with background
        if gt_text:
            draw_text_with_background(frame, gt_text, (10, 50), color)
        if pred_text:
            draw_text_with_background(frame, pred_text, (10, 100), color)
        
        # Write the frame to the output file
        out.write(frame)
        
        # Increment frame index
        frame_idx += 1
        print(f"Frame {frame_idx} processed.")
    
    # Release resources
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"Visualization saved to {output_path}")

def draw_text_with_background(img, text, position, color, font=cv2.FONT_HERSHEY_SIMPLEX, font_scale=1.25, thickness=2):
    """Draw text with a white background for better visibility."""
    text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]
    text_x, text_y = position
    box_coords = ((text_x, text_y - text_size[1] - 10), (text_x + text_size[0] + 10, text_y + 5))
    cv2.rectangle(img, box_coords[0], box_coords[1], (255, 255, 255), cv2.FILLED)
    cv2.putText(img, text, (text_x, text_y), font, font_scale, color, thickness, cv2.LINE_AA)

# Example usage
keysteps_dict = args.dataloader_params['keysteps']
keystep_id_to_desc = {i: v for i, v in enumerate(keysteps_dict.values())}

subject_results_dir = os.path.join(results_dir, 'subject_specific_results')

subject_trial = 'ms1_1'
csv_path = f'{subject_results_dir}/{subject_trial}.csv'  # Replace with the path to the generated CSV
video_path = '/standard/UVA-DSA/NIST EMS Project Data/EgoExoEMS_CVPR2025/Dataset/Final/ms1/cardiac_arrest/1/GoPro/GX010392_encoded_trimmed_deidentified.mp4'  # Replace with the path to the video
output_path = f'{subject_results_dir}/{subject_trial}_visualized.mp4'  # Output path for the visualized video

print(f"Visualizing keysteps for {subject_trial}...")
visualize_keysteps_from_csv(video_path, csv_path, output_path, keystep_id_to_desc)
print("Visualization complete.")


Visualizing keysteps for ms1_1...
Frame 1 processed.
Frame 2 processed.
Frame 3 processed.
Frame 4 processed.
Frame 5 processed.
Frame 6 processed.
Frame 7 processed.
Frame 8 processed.
Frame 9 processed.
Frame 10 processed.
Frame 11 processed.
Frame 12 processed.
Frame 13 processed.
Frame 14 processed.
Frame 15 processed.
Frame 16 processed.
Frame 17 processed.
Frame 18 processed.
Frame 19 processed.
Frame 20 processed.
Frame 21 processed.
Frame 22 processed.
Frame 23 processed.
Frame 24 processed.
Frame 25 processed.
Frame 26 processed.
Frame 27 processed.
Frame 28 processed.
Frame 29 processed.
Frame 30 processed.
Frame 31 processed.
Frame 32 processed.
Frame 33 processed.
Frame 34 processed.
Frame 35 processed.
Frame 36 processed.
Frame 37 processed.
Frame 38 processed.
Frame 39 processed.
Frame 40 processed.
Frame 41 processed.
Frame 42 processed.
Frame 43 processed.
Frame 44 processed.
Frame 45 processed.
Frame 46 processed.
Frame 47 processed.
Frame 48 processed.
Frame 49 proces